## Hospitalized



In [1]:
import pandas as pd
import pmdarima as pm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

aug_trainset = pd.read_csv('data/Anexos_7/training/total_hospitalized_augmented_training_data.csv')
testset = pd.read_csv('data/Anexos_7/test/total_hospitalized_test_data.csv')
original_trainset = pd.read_csv('data/Anexos_7/training/total_hospitalized_training_data.csv')
# Convert the 'Date' column to datetime format
aug_trainset['Date'] = pd.to_datetime(aug_trainset['Date'], format='%Y-%m-%d')
testset['Date'] = pd.to_datetime(testset['Date'], format='%Y-%m-%d')
original_trainset['Date'] = pd.to_datetime(original_trainset['Date'], format='%Y-%m-%d')


testset.set_index('Date', inplace=True)
aug_trainset.set_index('Date', inplace=True)
original_trainset.set_index('Date', inplace=True)

In [2]:
from sklearn.model_selection import ParameterGrid

def auto_arima_train(train, param_grid):
    d_from_diff = {}
    models = {}
    ignore_cols = ['Day', 'Month', 'Year', 'DayOfYear']
    provinces = [col for col in train.columns if col not in ignore_cols]
    
    for province in provinces:
        curr_train = train[[province]]
        
        # Grid search for hyperparameter tuning
        best_model = None
        best_params = {"seasonal": True, 'stepwise': False,"maxiter": 50, "out_of_sample": 0}
        best_aic = float('inf')
        
        for params in ParameterGrid(param_grid):
            model = pm.auto_arima(curr_train, 
                                  seasonal=params['seasonal'], 
                                  stepwise=params['stepwise'], 
                                  maxiter=params['maxiter'], 
                                  out_of_sample_size=params['out_of_sample'], 
                                  suppress_warnings=True)
            
            if model.aic() < best_aic:
                best_aic = model.aic()
                best_model = model
                best_params['seasonal'] = params['seasonal']
                best_params['stepwise'] = params['stepwise']
                best_params['maxiter'] = params['maxiter']
                best_params['out_of_sample'] = params['out_of_sample']
        
        models[province] = best_model

        # Check for stationarity and apply differencing if needed
        d = 0
        result = adfuller(curr_train[province])
        while result[1] > 0.05:
            curr_train = curr_train.diff().dropna()
            d += 1
            result = adfuller(curr_train[province])
        d_from_diff[province] = d
        print(f"Best parameters for province '{province}': (d={d} from differencing, d={best_model.order[1]} d from AutoArima)\n")
        for p in best_params:
            print(f'{p}: {best_params[p]}')
        # Create subplots for ACF and PACF
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        plot_acf(curr_train[province], ax=axes[0])
        plot_pacf(curr_train[province], ax=axes[1])
        axes[0].set_title(f'ACF for {province} (MA={best_model.order[2]})')
        axes[1].set_title(f'PACF for {province} (AR={best_model.order[0]})')
        plt.tight_layout()
        plt.show()
    
    return models, d_from_diff





def evaluate_model(models, d_from_diff, train, test):
    for province in models.keys():
        auto_model = models[province]
        curr_test = test[province]
        curr_train = train[province]
        auto_predictions = auto_model.predict(n_periods=len(curr_test))
        d = d_from_diff[province]
    # Fit ARIMA model on the training set
        model = ARIMA(curr_train, order = (auto_model.order[0], d, auto_model.order[2]),suppress_warnings=True)
        model_fit = model.fit()
        # Make predictions on the test set
        predictions = model_fit.forecast(steps=len(curr_test))
        print(f' AutoArima Predictions for {province}: incomes. d = {auto_model.order[1]}') 
        print(auto_predictions)
        print()
        print(f'Arima Predictions for {province}: incomes. d = {d}') 
        print(predictions)


        # Calculate evaluation metrics
        auto_mae = mean_absolute_error(curr_test, auto_predictions)
        auto_mse = mean_squared_error(curr_test, auto_predictions)
        auto_rmse = np.sqrt(auto_mse)
        auto_mape = np.mean(np.abs((curr_test - auto_predictions) / curr_test)) * 100
        auto_nmse = auto_mse / np.var(curr_test)
        # Calculate evaluation metrics
        mae = mean_absolute_error(curr_test, predictions)
        mse = mean_squared_error(curr_test, predictions)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((curr_test - predictions) / curr_test)) * 100
        nmse = mse / np.var(curr_test)

        print(f'MAE for autoArima: {auto_mae}')
        print(f'MSE for autoArima: {auto_mse}')
        print(f'RMSE for autoArima: {auto_rmse}')
        print(f'MAPE for autoArima: {auto_mape}%')
        print(f'NMSEfor autoArima: {auto_nmse}')

        print()
        print(f'MAE for ARIMA: {mae}')
        print(f'MSE for ARIMA: {mse}')
        print(f'RMSE for ARIMA: {rmse}')
        print(f'MAPE for ARIMA: {mape}%')
        print(f'NMSE for ARIMA: {nmse}')


        # Plot prediction against test and training trends
        
        plt.figure(figsize=(12,8))
        plt.title(f"Predictions for {province}")
        plt.plot(curr_train, label="Training")
        plt.plot(curr_test, label="Test")
        plt.plot(curr_test.index, auto_predictions, label="AutoArima Predicted", color = 'green')
        plt.plot(curr_test.index, predictions, label='Arima Predicted', color='red')
        plt.legend()
        plt.show()




        # # Plot actual vs predicted values
        # plt.figure(figsize=(10, 6))
        # plt.plot(curr_test.index, curr_test, 'o', label='Actual') # Use 'o' for markers 
        # plt.plot(curr_test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
        # plt.legend()
        # plt.show()

        # # Plot actual vs predicted values
        # plt.figure(figsize=(10, 6))
        # plt.plot(curr_test.index, curr_test, 'o', label='Actual') # Use 'o' for markers 
        # plt.plot(curr_test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
        # plt.legend()
        # plt.show()
        print(model_fit.summary()) 



In [ ]:
# Define the parameter grid
param_grid = {
    'seasonal': [True, False],
    'stepwise': [True, False],
    'maxiter': [50, 70,80,100],
    'out_of_sample': [0, 10,20]
}

models, d_from_diff = auto_arima_train(original_trainset, param_grid)

In [ ]:
evaluate_model(models, d_from_diff, original_trainset, testset)

In [ ]:
models, d_from_diff = auto_arima_train(aug_trainset, param_grid)

In [ ]:
evaluate_model(models, d_from_diff, aug_trainset, testset)